<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Attention_Networks_(GAT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dgl torchdata

In [ ]:
import dgl
from dgl.nn import GATConv
import torch.nn as nn
import torch.nn.functional as F
import torch

# Define the GAT model
class GAT(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_heads):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_feats, hidden_feats, num_heads=num_heads)
        self.gat2 = GATConv(hidden_feats * num_heads, out_feats, num_heads=num_heads)

    def forward(self, g, x):
        x = self.gat1(g, x)
        x = F.elu(x)
        x = x.view(-1, x.shape[1] * x.shape[2])
        x = self.gat2(g, x)
        return x.mean(1)

# Example usage
g = dgl.graph(([0, 1, 2], [1, 2, 0]))
features = torch.tensor([[1.], [2.], [3.]])
model = GAT(1, 4, 1, 2)
output = model(g, features)
print(output)